# Выбор локации для скважины

# Описание проекта

Допустим, мы работаем в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Нужно построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.


**`Также нам даны условия задачи:`**

* Для обучения модели подходит только `линейная регрессия` (остальные — недостаточно предсказуемые);
* При разведке региона исследуют `500 точек`, из которых с помощью машинного обучения `выбирают 200 лучших` для разработки;
* `Бюджет` на разработку скважин в регионе — `10 млрд рублей`;
* При нынешних ценах `один баррель` сырья приносит `450 рублей дохода`. `Доход с` каждой `единицы` продукта составляет `450 тыс.` рублей, поскольку объём указан в тысячах баррелей;
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков `меньше 2.5%`. Среди них `выбирают` регион с `наибольшей средней прибылью`

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
from scipy import stats as st
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
data1 = pd.read_csv('/datasets/geo_data_0.csv')
data2 = pd.read_csv('/datasets/geo_data_1.csv')
data3 = pd.read_csv('/datasets/geo_data_2.csv')
data1.head()
#data2.head()
#data3.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


**Описание данных**

* `id` — уникальный идентификатор скважины;
* `f0, f1, f2` — три признака точек (неважно, что они означают, но сами признаки значимы);
* `product` — объём запасов в скважине (тыс. баррелей)

Так как нам даны данные трех разных регионов и их описание данных в названиях колонок схожи, будет неудобно объединять датасеты вместе

* **Построим для них функцию, которая выведет информацию о всех трех регионах:**

In [3]:
def my_info(x, y):  # аргумент x примет значение сохраненного датасета в переменной;
                    # аргумент y примет описательный характер датасета, по которому будет описываться информация
    print(y)
    display(x.head())
    print(f'Размер {y}: {x.shape}')
    print()
    print(f'Информация о {y}:')
    print()
    display(x.info())
    display(x.describe())
    print(f'Корреляция между признаками в {y}:')
    display(x.corr())
    print(f'Кол-во пропущенных значений в {y}:') 
    display(x.isna().sum())
    print()
    print(f'Кол-во дубликатов в {y}: {x.duplicated().sum()}')

In [4]:
my_info(data1, 'data1')

data1


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Размер data1: (100000, 5)

Информация о data1:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


Корреляция между признаками в data1:


,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


Кол-во пропущенных значений в data1:


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Кол-во дубликатов в data1: 0


In [5]:
my_info(data2, 'data2')

data2


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


Размер data2: (100000, 5)

Информация о data2:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


Корреляция между признаками в data2:


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Кол-во пропущенных значений в data2:


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Кол-во дубликатов в data2: 0


In [6]:
my_info(data3, 'data3')

data3


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Размер data3: (100000, 5)

Информация о data3:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


Корреляция между признаками в data3:


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Кол-во пропущенных значений в data3:


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Кол-во дубликатов в data3: 0


Фиксируем, что нет пропусков и дубликатов, но проверим еще на уникальные значения в колонках

In [7]:
data1['id'].value_counts().to_frame()
#data1['f0'].value_counts().to_frame()
#data1['f1'].value_counts().to_frame()
#data1['f2'].value_counts().to_frame()

,id
HZww2,2
AGS9W,2
A5aEY,2
fiKDv,2
bsk9y,2
...,...
9XAqq,1
oFIXm,1
fr563,1
crrAM,1


In [8]:
data1[data1['id'] == 'HZww2']
data1[data1['id'] == 'A5aEY']

,id,f0,f1,f2,product
3389,A5aEY,-0.039949,0.156872,0.209861,89.249364
51970,A5aEY,-0.180335,0.935548,-2.094773,33.020205


* Видим, что в столбце с `"id"` есть **дубликаты**, то есть скважины с одинаковыми `id`. Но так как сами `признаки отличаются`, то они могут принести **пользу** для модели. Оставляем их

Проверим их доли в датасетах

In [9]:
dup1 = data1['id'].duplicated().mean()
print(f'Доля дубликатов в колонке \'id\' в data1 составляет: {dup1:.2%}')

Доля дубликатов в колонке 'id' в data1 составляет: 0.01%


In [10]:
dup2 = data2['id'].duplicated().mean()
print(f'Доля дубликатов в колонке \'id\' в data2 составляет: {dup2:.3%}')

Доля дубликатов в колонке 'id' в data2 составляет: 0.004%


In [11]:
dup3 = data3['id'].duplicated().mean()
print(f'Доля дубликатов в колонке \'id\' в data3 составляет: {dup3:.3%}')

Доля дубликатов в колонке 'id' в data3 составляет: 0.004%


Эти доли ничтожны малы во всех регионах. Как было ранее сказано, их стоит оставить, так как признаки с одинаковыми **id** разные.

* Взглянем еще раз на корреляции в трех регионах: 

In [12]:
print('Корреляция между признаками в первом регионе (data1):')
display(data1.corr())

print('Корреляция между признаками во втором регионе (data2):')
display(data2.corr())

print('Корреляция между признаками в третьем регионе (data3):')
display(data3.corr())

Корреляция между признаками в первом регионе (data1):


,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


Корреляция между признаками во втором регионе (data2):


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Корреляция между признаками в третьем регионе (data3):


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


**Вывод по корреляциям в регионах:**

* Замечаем во втором регионе **data2** **`сильную корреляцию`** между `f2` признаком и целевым признаком `product`. Возможно, эта сильная зависимость из-за того, что данные выдуманы - синтетические. Все остальные признаки во втором регионе `слабо` коррелируют между собой (`f2 с f0, f1 отрицательно коррелируют` друг с другом, целевой признак также отрицательную корреляцию имеет с `f0, f1`);


* В первом и третьем регионах **(data1, data3)** немного схожи. Целевой признак **product** с **f** имеет `низкую положительную` зависимость. В третьем регионе **(data3)** очень слабые корреляции (за исключением `целевого признака и f2`)


* В первом регионе **(data1)** тоже относительно очень слабые корреляции между собой, пожалуй, за исключением `f0, f1`. У них `слабо отрицательная` корреляция между собой

### Вывод

* Были загружены данные и с помощью фунции разобраны датасеты:
  * Типы колонок корректны;
  * Были проверены данные на дубликаты. Полных дубликатов не оказалось, но с помощью метода `value_counts()` выявили дубликаты по колонке с **'id'**, которые было решено оставить, так как могут помочь принести пользу модели;
  * Изучили корреляции для данных по каждому региону;
  * Данные соответсвуют описанию;
  * Пропуски в данных отсутствуют 

## Обучение и проверка модели

**`Создаем функцию для вычисления среднего запаса предсказанного сырья для каждого региона и RMSE модели`**

* Функция обучает модель Линейной Регрессии, считает предсказания и ответы на валидационной выборке, и возвращает средний запас предсказанного сырья для каждого региона **`(predict_mean)`** и RMSE **`(rmse)`** модели (`также целевой признак на валидационной выборке и сами предсказания для дальнейшего выявления прибыли`)

In [13]:
def feat_tar_model(x, data): # аргумент x берет значение выборки, аргумент несет описательный характер для каждого региона 
    features = x.drop(['product', 'id'], axis=1) # создаем признаки и целевой признак для каждой выборки отдельного региона 
    target = x['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, 
                                                                                  random_state=12345)
    # разделяем на обучающую и валидационные выборки 
    
    print(f'Train выборка {data}: {features_train.shape}, {target_train.shape}') # проверяем разделение/размер выборок
    print(f'Valid выборка {data}: {features_valid.shape}, {target_valid.shape}')
    print()
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    
    predict_mean = predict.mean()
    mse = mean_squared_error(target_valid, predict)
    rmse = np.sqrt(mse)
    
    print(f'Cредний запас предсказанного сырья для {data} региона: {round(predict_mean, 3)}')
    print(f'RMSE модели {data}: {round(rmse, 3)}')
    
    return predict_mean, rmse, predict, target_valid

* Сохраняем в переменных среднее сырье, RMSE модели, а также предсказания и ответы **для каждого отдельного региона**

In [14]:
predict_mean1, rmse1, predict1, target_valid1 = feat_tar_model(data1, 'data1') # для первого региона data1

Train выборка data1: (75000, 3), (75000,)
Valid выборка data1: (25000, 3), (25000,)

Cредний запас предсказанного сырья для data1 региона: 92.593
RMSE модели data1: 37.579


In [15]:
predict_mean2, rmse2, predict2, target_valid2 = feat_tar_model(data2, 'data2') # для второго региона data2

Train выборка data2: (75000, 3), (75000,)
Valid выборка data2: (25000, 3), (25000,)

Cредний запас предсказанного сырья для data2 региона: 68.729
RMSE модели data2: 0.893


In [16]:
predict_mean3, rmse3, predict3, target_valid3 = feat_tar_model(data3, 'data3') # для третьего региона data2

Train выборка data3: (75000, 3), (75000,)
Valid выборка data3: (25000, 3), (25000,)

Cредний запас предсказанного сырья для data3 региона: 94.965
RMSE модели data3: 40.03


### Вывод

* Для региона 1 и 3 показатели схожи что в **среднем запасе** сырья (`92.593 и 94.965` соответственно), что в средней квадратичной ошибки корня **(rmse)** (`37.579` и `40.03` соответственно). Большие показатели **RMSE** в этих регионах свидельсвтуют о неточности модели, как сильно правильный ответ отличается от предсказаний
* Для второго региона **(data2)** модель показывает обратное **RMSE** `0.893`, самое низкое среди других регионах. Это говорит о точности предсказаний, совпадении с правильными ответами и качество модели. `НО!` Это происходит за счет того, что корреляция с целевым признаком **product** и признаком **f2** очень сильная, прямая

## Подготовка к расчёту прибыли

Обращаясь к условиям задачи, а именно:

* При разведке региона исследуют `500 точек`, из которых с помощью машинного обучения `выбирают 200 лучших` для разработки;
* `Бюджет` на разработку скважин в регионе — `10 млрд рублей`;
* При нынешних ценах `один баррель` сырья приносит `450 рублей дохода`. `Доход с` каждой `единицы` продукта составляет `450 тыс.` рублей, поскольку объём указан в тысячах баррелей

Cоздадим соответсвтенные переменные для расчета прибыли

In [17]:
BEST_OF_WELLS = 200 # лучшик точки для разработки 
BUDGET = 10**10 # в десятой степени обозначаем 10 млрд
REVENUE_PER_BARREL = 450000 # доход с каждой единицы

FOR_ONE_WELL = BUDGET / BEST_OF_WELLS
print(f'Бюджет на одну скважину: {FOR_ONE_WELL}')

BREAK_EVEN_WELL = FOR_ONE_WELL / REVENUE_PER_BARREL # объем на одну скважину
print(f'Достаточный объем сырья для безубыточной разработки новой скважины составляет: {round(BREAK_EVEN_WELL, 2)}')

Бюджет на одну скважину: 50000000.0
Достаточный объем сырья для безубыточной разработки новой скважины составляет: 111.11


Напомним наши средние показатели в каждом из регионов

In [18]:
region_predict = [predict_mean1, predict_mean2, predict_mean3]

for i in region_predict:
    if i >= BREAK_EVEN_WELL: 
        print('Здесь достаточно для безубыточной разработки новой скважины')
    else:
        print(f'Прогнозируется убыток, так как добыча сырья составляет {round(i, 2)} по региону')
            
print()
print(f'Средний запас сырья в 1-ом регионе: {round(predict_mean1, 2)}')
print(f'Средний запас сырья в 2-ом регионе: {round(predict_mean2, 2)}')
print(f'Средний запас сырья в 3-ем регионе: {round(predict_mean3, 2)}')

Прогнозируется убыток, так как добыча сырья составляет 92.59 по региону
Прогнозируется убыток, так как добыча сырья составляет 68.73 по региону
Прогнозируется убыток, так как добыча сырья составляет 94.97 по региону

Средний запас сырья в 1-ом регионе: 92.59
Средний запас сырья в 2-ом регионе: 68.73
Средний запас сырья в 3-ем регионе: 94.97


### Вывод

* Рассчитали **объем сырья**, который требуется на разработку новой скважины **без убытка**. Она составила **111.11**
* Что касается по среднему запасу сырья в трех регионах, все они `являются убыточными` для этого расчета (**92.59, 68.73 и 94.97** соответственно)

## Расчёт прибыли и рисков

* Создадим функцию, которая будет возвращать расчёт прибыли по выбранным скважинам и предсказаниям модели

In [19]:
def profit(target_valid, predict): # в качестве аргументов функция берет целевой признак на валидационной выборке и предсказания
    target_valid = pd.Series(target_valid).reset_index(drop=True) # создаем целевые значения с индексами, 
                                                                  # чтобы их перебирать в дальнейшем
        
    predict = pd.Series(predict).reset_index(drop=True) #создаем предсказания по индексам,чтобы можно также было отсортировать  
    
    predict_sort = predict.sort_values(ascending=False) # фиксируем в порядке убывания предсказания, то есть от макс. к мини
    result = target_valid[predict_sort.index][:200] # выбираем целевое значение соотвествующее индексам предсказаний объема сырья
                                                    # в 200 лучших точках (т.к. было ранее отсортировано)
    summ = result.sum() # суммируем его
    
    return REVENUE_PER_BARREL * summ - BUDGET # возвращаем прибыль для полученного сырья

In [20]:
profit1 = profit(target_valid1, predict1)
print(f'Прибыль для полученного объема сырья в 200 лучших точках основанная на предсказанном результате для 1-го региона: {profit1}')

Прибыль для полученного объема сырья в 200 лучших точках основанная на предсказанном результате для 1-го региона: 3320826043.1398506


In [21]:
profit2 = profit(target_valid2, predict2)
print(f'Прибыль для полученного объема сырья в 200 лучших точках основанная на предсказанном результате для 2-го региона: {profit2}')

Прибыль для полученного объема сырья в 200 лучших точках основанная на предсказанном результате для 2-го региона: 2415086696.681511


In [22]:
profit3 = profit(target_valid3, predict3)
print(f'Прибыль для полученного объема сырья в 200 лучших точках основанная на предсказанном результате для 3-го региона: {profit3}')

Прибыль для полученного объема сырья в 200 лучших точках основанная на предсказанном результате для 3-го региона: 2710349963.5998325


### Вывод

* По прибыли лидирует 1-ый регион **(data1)**, хотя у него было чуть меньше предсказанного сырья, чем у 3-го региона (data3). У него **(data1)**  **`3 млрд 320 млн 826043 тыс`**;
* Затем следует 3-ий регион **(data3)**. У него примерно **2 млрд 710 млн 349963 тыс**;
* И на последнем месте по прибыли идет 2-ой регион. У него примерно **2 млрд 415 млн 86697 тыс**

### Риски убытков, средняя прибыль и доверительный интервал

In [23]:
state = np.random.RandomState(12345) # задаем переменную, чтобы каждый раз менялся на случайность 

def bootstrap(target_valid, predict, data): # функция принимает ответы, предсказания и описатлеьны характер отдельного региона
    values = [] # в пустой список будем помещать значение функции на подвыборках
    
    for i in range(1000): 
        target_subsample = pd.Series(target_valid).reset_index(drop=True).sample(n=500, replace=True, #n-число скважин
                                                                                 random_state=state) 
        predict = pd.Series(predict).reset_index(drop=True)
        predict_subsample = predict[target_subsample.index]
        
        values.append(profit(target_subsample, predict_subsample))
        
    values = pd.Series(values) # прибыль на бустрепе в 1000 диапазоне
    lower = values.quantile(0.025) # вычисляем 97.5% значений, которые будут принимать не меньше этого значения
    values_mean = values.mean() # считаем среднюю прибыль 
        
    #confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())
    
    print(f'Средняя выручка {data} региона: {round(values_mean, 3)}')
    print(f'95% доверительный интервал {data} региона: от {round(values.quantile(0.025),2)} до {round(values.quantile(0.975),2)}')
    print(f'2.5%-квантиль: {lower}')
    print(f'Риски убытков: {round((values < 0).mean() * 100, 2)}%')# вычисляем процент убытка (Убыток — 
                                                                 # это отрицательная прибыль)

In [24]:
bootstrap(target_valid1, predict1, 'data1')

Средняя выручка data1 региона: 396164984.802
95% доверительный интервал data1 региона: от -111215545.89 до 909766941.55
2.5%-квантиль: -111215545.89049526
Риски убытков: 6.9%


In [25]:
bootstrap(target_valid2, predict2, 'data2')

Средняя выручка data2 региона: 461155817.277
95% доверительный интервал data2 региона: от 78050810.75 до 862952060.26
2.5%-квантиль: 78050810.7517417
Риски убытков: 0.7%


In [26]:
bootstrap(target_valid3, predict3, 'data3')

Средняя выручка data3 региона: 392950475.171
95% доверительный интервал data3 региона: от -112227625.38 до 934562914.55
2.5%-квантиль: -112227625.37857565
Риски убытков: 6.5%


### Вывод

* Рассчитали среднюю прибыль для каждого региона и другие необходимые параметры для выявления регионов, где риски будут составлять не менее 2.5%:


* В **первом** регионе **data1**:
  * **риски убытков** оказались наибольшими - **`6.9%`**;
  * **средняя выручка** на втором месте среди регионов - **396 млн 164985 тыс**;
  * **95% значений находятся** `от -111215545.89 до 909766941.55`
* Во **втором** регионе **data2**:
  * **риски убытков** самые малые среди других регионов - **`0.7%`**;
  * **средняя выручка** самая большая - **461 млн 155817 тыс**;
  * **95% значений находятся** `от 78050810.75 до 862952060.26 тыс`
* В **третьем** регионе **data3**:
  * **риски убытков** чуть меньше, чем в первом регионе - **`6.5%`**;
  * **средняя выручка** наименьшая среди региново - **392 млн 950475 тыс**;
  * **95% значений находятся** `от -112227625.38 до 934562914.55`
  
## Вывод

* Данные были загружены и разобраны;
* Были выявлены некоторые дубликаты, которая были оставлены, так как они были полезны для модели;
* Рассмотрели корреляцию в регионах;
* Была создана функция, которая помогла разделить на выборки, обучить модель, посчитать средний объем сырья в каждом регионе и показать **RMSE** модели, которая показывает качество модели;
* Необходимый объем для безубыточной разработки новой скважины оказалось значение в **111.11 тыс. баррелей**;
* Во всех трех регионах **`объем предсказанного сырья`** оказался **меньше** - **92.59, 68.73 и 94.97** соответственно;
* Посчитали среднюю прибыль по выбранным скважинам (200 луших) и предсказаниям модели;
* Техников **Bootstrap** были выбраны случайно 500 точек, из которых 200 лучших из них выявлены для подсчета средней прибыли, выявления 95%-го доверительного интервала и подсчета риска убытков 

Среди трех регионов, исходя из расчетов, **`был предложен регион под номером 2 (data2) для разработки скважин`**:
  * Здесь оказались `риски убытков меньше 2.5%` - а именно 0.7%;
  * **средняя выручка** самая большая - **461 млн 155817 тыс;**
  * **95% значений находятся** `от 78050810.75 до 862952060.26 тыс`
  * также **97.5% значений** составляет дохода **не меньше, чем 78 млн 050810.7 тыс**